Jusqu'à présent, je me suis concentré sur le fichier train.csv mais il y a aussi authorData qui n'a juste pas l'embedding.

J'avais vu que l'embedding n'était pas méga importante mais je vais chercher à la garder. Pour cela, je vais chercher à estimer lesparamètres de l'embedding.

(Notations : D1 = train.csv; D2 = authorData.csv; N = colonnes non-embedding(=explicites parce que je les connais presque qu'elles sont presques lisibles par un humain); M = colonnes embeddées.)

idée : utiliser train pour apprendre l'embedding sur N colonnes pour en prédire m<=M.
M=1024 donc prédire les 1024 est violent. On se propose de prédire les m termes de la pca

Etapes (d'entrainement et calibration du modèle)
* PCA(m) sur M colonnes de train
* diff train, test et X, Y 
* prédiction(m) à partir de N de X_train
  * modèle
  * métrique

In [1]:
import pandas as pd

from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

from notebook_helper import explicit_part_preparation, explicit_part_transform_from_processed_model, pca_transform

In [2]:
df1 = pd.read_csv("./data/train.csv", sep=",")
df2 = pd.read_csv("./data/authorData.csv", sep=",")

### a. PCA

In [3]:
m = 20
df1_reduced = pca_transform(df1, n_components=m)

In [4]:
df1_reduced.head(3)

,engagement,timestamp,language,feature1,feature2,followers,author,word_count,shared_url_count,shared_url_domain,...,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20
0,15,1655784230254,en,73,0,696,bd20432d80dfe4825a7a106312bda52e,18,0,NaN,...,-0.009567,-0.125106,0.033840,0.020685,-0.051285,-0.001273,-0.021936,0.047203,-0.027051,-0.038264
1,91007,1655599613254,en,100,5,17526,71e2ecc6cc1d6515a6b122fab4c63bfc,15,0,NaN,...,-0.098711,0.001065,0.018272,0.008464,-0.063585,-0.009290,0.011504,0.089991,-0.043554,0.025581
2,3,1655788872254,en,0,-5,239,2a9109f38d90a2a96e284a07c1a57e9b,8,0,NaN,...,-0.026528,-0.009448,0.066015,-0.002279,-0.005414,-0.017258,-0.010806,-0.032932,-0.051068,-0.004298


### b. préparation des données

In [5]:
df1_prepared, preparation_model = explicit_part_preparation(df1_reduced)
df2_prepared = explicit_part_transform_from_processed_model(df2, preparation_model)

In [6]:
df1_prepared.head(3)

,engagement,timestamp,feature2,followers,word_count,is_reply,contains_video,contains_image,V1,V2,...,shared_url_extension_net,shared_url_extension_news,shared_url_extension_org,shared_url_extension_other,shared_url_extension_tv,is_shared_url,month_post,day_of_week_post,hour_post,feature1_is_max
0,1.176091,6.727053,0,2.842609,1.278754,False,False,True,0.040117,0.039279,...,0.0,0.0,0.0,1.0,0.0,False,6,1,4,False
1,4.959075,8.278642,5,4.243683,1.204120,False,False,False,-0.154545,0.130231,...,0.0,0.0,0.0,1.0,0.0,False,6,6,0,True
2,0.477121,5.840107,-5,2.378398,0.954243,True,False,False,0.319861,-0.050030,...,0.0,0.0,0.0,1.0,0.0,False,6,1,5,False


In [7]:
df2_prepared.head(3)

,engagement,timestamp,feature2,followers,word_count,is_reply,contains_video,contains_image,shared_url_radical_amazon,shared_url_radical_breitbart,...,shared_url_extension_net,shared_url_extension_news,shared_url_extension_org,shared_url_extension_other,shared_url_extension_tv,is_shared_url,month_post,day_of_week_post,hour_post,feature1_is_max
0,0.000000,7.881288,0,2.999565,1.414973,False,True,False,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,False,6,0,8,True
1,0.000000,7.666555,-5,0.698970,1.000000,False,False,False,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,False,6,0,16,True
2,3.542327,8.595291,-5,5.657622,1.662758,False,False,False,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,False,6,3,16,True


### c. entraînement N->m

In [8]:
V_cols = [col for col in df1_prepared.columns if "V" in col]

# Separation of the target
X = df1_prepared.drop(columns=V_cols)
y = df1_prepared[V_cols]

# Normalisation
sc_X, sc_y = StandardScaler(), StandardScaler()
X_train, y_train = sc_X.fit_transform(X), sc_y.fit_transform(y)
X_eval = sc_X.transform(df2_prepared)

# Model training
base = lgb.LGBMRegressor(
    objective='regression',
    random_state=42,
)

model = MultiOutputRegressor(base)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_eval)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001913 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 891
[LightGBM] [Info] Number of data points in the train set: 4150, number of used features: 16
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000710 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 891
[LightGBM] [Info] Number of data points in the train set: 4150, number of used features: 16
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000745 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, 

c:\Users\mathi\dev\ml2025\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\mathi\dev\ml2025\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\mathi\dev\ml2025\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\mathi\dev\ml2025\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\mathi\dev\ml2025\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Us

In [9]:
df2_completed = pd.concat(
    [
        df2_prepared.reset_index(drop=True),
        pd.DataFrame(y_pred, columns=V_cols)
    ],
    axis=1,
)

In [23]:
nb_cols_difference_after_preparation = df2.shape[0] - df2_completed.shape[0]
nb_cols_outliers_followers_and_word_count = df2.query("followers == 0 or word_count == 0").shape[0]

print(
    f"{df2_completed.shape[0]} lignes après préparation\n{df2.shape[0]} lignes "
    f"avant préparation.\nOn en a perdu {nb_cols_difference_after_preparation}\n"
    f"On a retiré {nb_cols_outliers_followers_and_word_count} outliers et \n"
    f"df2_completed.shape[0] + nb_cols_outliers_followers_and_word_count = {df2_completed.shape[0]} + {nb_cols_outliers_followers_and_word_count} = {df2_completed.shape[0] + nb_cols_outliers_followers_and_word_count}"
    f" {"=" * (df2_completed.shape[0] + nb_cols_outliers_followers_and_word_count == df2.shape[0]) + "≠" * (df2_completed.shape[0] + nb_cols_outliers_followers_and_word_count != df2.shape[0])} "
    f"{df2.shape[0]}"
)

626063 lignes après préparation
629877 lignes avant préparation.
On en a perdu 3814
On a retiré 3814 outliers et 
df2_completed.shape[0] + nb_cols_outliers_followers_and_word_count = 626063 + 3814 = 629877 = 629877


In [27]:
df1_prepared.to_pickle("./data/train_d1.pickle")
df2_completed.to_pickle("./data/train_d2.pickle")